## INSTALAR LAS DEPENDENCIAS NECESARIAS PARA USAR EL NOTEBOOK

In [ ]:
!pip install numpy pandas matplotlib python-dotenv requests 

## IMPORTAR LIBRERIAS

In [ ]:
import os
from dotenv import load_dotenv
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt

## CARGAR CREDENCIALES DESDE UN ARCHIVO .ENV
Deberas crear un archivo ".env" en el mismo directorio de este documento. Alli cargar tus credenciales como
IOL_USERNAME=TuUsuarioDeIOL
IOL_PASSWORD=TuContraseñaDeIOL

In [ ]:
load_dotenv()
IOL_USERNAME = os.getenv("IOL_USERNAME")
IOL_PASSWORD = os.getenv("IOL_PASSWORD")
#Levanto error si falla alguna credencial
if not IOL_USERNAME or not IOL_PASSWORD:
    raise ValueError("Faltan las credenciales en el archivo .env")

## AUTENTICACION Y MANEJO DE TOKENS
Bearer Token: Es el token principal que se usa para autenticar cada request. Tiene una validez limitada (15 minutos en este caso).
Refresh Token: Permite renovar el bearer token sin necesidad de proporcionar nuevamente el usuario y contraseña. Este método mejora la seguridad y reduce el uso indebido de credenciales.
Renovación Automática: El código detecta si el bearer token ha expirado y automáticamente lo renueva utilizando el refresh token.

In [ ]:
BASE_URL = "https://api.invertironline.com"
TOKEN_URL = f"{BASE_URL}/token"

class InvertirOnlineAPI:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.bearer_token = None
        self.refresh_token = None
        self.token_expiry = None

#Autenticar al usuario y obtener tokens
    def authenticate(self):
        data = {
            "username": self.username,
            "password": self.password,
            "grant_type": "password"
        }
        response = requests.post(TOKEN_URL, data=data)
        response.raise_for_status()
        tokens = response.json()
        self.bearer_token = tokens["access_token"]
        self.refresh_token = tokens["refresh_token"]
        self.token_expiry = time.time() + tokens["expires_in"] - 60  #Margen de seguridad
        print("Autenticación exitosa. Bearer token obtenido.")

#Refrescar el bearer token
    def refresh_bearer_token(self):
        if not self.refresh_token:
            raise ValueError("Refresh token no disponible. Autentica primero.")
        data = {
            "refresh_token": self.refresh_token,
            "grant_type": "refresh_token"
        }
        response = requests.post(TOKEN_URL, data=data)
        response.raise_for_status()
        tokens = response.json()
        self.bearer_token = tokens["access_token"]
        self.token_expiry = time.time() + tokens["expires_in"] - 60
        print("Bearer token renovado.")

#Devolver los headers
    def get_headers(self):
        if not self.bearer_token or time.time() > self.token_expiry:
            print("Token expirado o no disponible. Renovando...")
            self.refresh_bearer_token()
        return {
            "Authorization": f"Bearer {self.bearer_token}"
        }

#Crear instancia de la API
iol_api = InvertirOnlineAPI(IOL_USERNAME, IOL_PASSWORD)
iol_api.authenticate()

## COTIZACION DOLAR MEP

In [ ]:
#Obtener el valor del USD MEP, por arguumento pasar el simbolo del activo a usar para el calculo
#Devuelve float del USD MEP
def obtener_dolar_mep(simbolo):
    endpoint = f"{BASE_URL}/api/v2/Cotizaciones/MEP/{simbolo}"
    headers = iol_api.get_headers()
    #Realizar la solicitud GET al endpoint
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()  # Levanta excepción si ocurre un error
    #Obtener la respuesta en formato JSON
    cotizacion_data = response.json()
    # Si la respuesta es un float directamente, asignamos a dolar_mep
    if isinstance(cotizacion_data, float):
        dolar_mep = cotizacion_data
    else:
        # Si la respuesta es un diccionario, extraemos el valor como antes
        dolar_mep = cotizacion_data.get("valor", None)
    return dolar_mep

#Obtener el valor del Dólar MEP para el símbolo AL30
simbolo = "AL30"
dolar_mep = obtener_dolar_mep(simbolo)

#Imprimo el valor del dolar MEP
print(f"El valor del Dólar MEP para el símbolo {simbolo} es: {dolar_mep}")

## ESTADO DE CUENTA SIMPLIFICADO
Devuelve dataframe con cuentas, moneda de la misma, saldos disponibles, valorizacion de titulos y total

In [ ]:
#Obtengo el estado de cuenta del usario, devuelve diccionario con informacion
def get_estado_cuenta():
    endpoint = f"{BASE_URL}/api/v2/estadocuenta"
    headers = iol_api.get_headers()
    # Realizar la solicitud GET al endpoint
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()  # Levanta excepción si ocurre un error
    # Devuelve los datos como un diccionario
    return response.json()

#Llamo a la funcion
estado_cuenta = get_estado_cuenta()

#Proceso la informacion, acepta por argumento diccionario para el estado de cuenta, devuelve pd.DataFrame
def procesar_estado_cuenta(data):
    cuentas = data.get("cuentas", [])
    #Extraigo las columnas relevantes
    procesado = [
        {
            "Número de Cuenta": cuenta.get("numero"),
            "Tipo": cuenta.get("tipo"),
            "Moneda": cuenta.get("moneda"),
            "Saldo Disponible": cuenta.get("saldo"),
            "Títulos Valorizados": cuenta.get("titulosValorizados"),
            "Total": cuenta.get("total")
        }
        for cuenta in cuentas
    ]
    return pd.DataFrame(procesado)

#Configura pandas para no usar notación científica
pd.set_option("display.float_format", "{:.2f}".format)
#Procesar los datos
df_estado_cuenta = procesar_estado_cuenta(estado_cuenta)
#Mostrar el DataFrame
print("Estado de Cuenta Procesado:")
df_estado_cuenta

## PARA VER LA TOTALIDAD DE LOS ACTIVOS

In [ ]:
#Obtengo el portafolio para el pais especificado  lo proceso. El pais se pasa por argumento y toma el valor del UUSD MEP antes usado
#Devuelve pd.DataFrame con el portafolio
def obtener_y_procesar_portafolio(pais, dolar_mep):
    # Endpoint para obtener el portafolio
    endpoint = f"{BASE_URL}/api/v2/portafolio/{pais}"
    headers = iol_api.get_headers()
    # Solicitar datos de portafolio a la API
    response = requests.get(endpoint, headers=headers)
    response.raise_for_status()  # Levantar error si la solicitud falla
    portafolio_data = response.json()
    # Procesar los activos del portafolio
    activos = portafolio_data.get("activos", [])
    procesado = [
        {
            "Simbolo": activo["titulo"].get("simbolo"),
            "Descripcion": activo["titulo"].get("descripcion"),
            "Cantidad": activo.get("cantidad"),
            "Ultimo Precio": activo.get("ultimoPrecio"),
            "PPC": activo.get("ppc"),
            "Ganancia Porcentaje": activo.get("gananciaPorcentaje"),
            "Ganancia en Dinero": activo.get("gananciaDinero"),
            "Valorizado": activo.get("valorizado"),
            "Tipo de Activo": activo["titulo"].get("tipo"),
            "Mercado": activo["titulo"].get("mercado"),
            "Moneda": activo["titulo"].get("moneda"),
            "Valorizado USD MEP": activo.get("valorizado") / dolar_mep
            if activo["titulo"].get("moneda") == "peso_Argentino"
            else activo.get("valorizado"),
        }
        for activo in activos
    ]

    # Convertir la lista procesada en un DataFrame
    return pd.DataFrame(procesado)

#Llamo a la funcion
pais = "Argentina"
df_portafolio_procesado = obtener_y_procesar_portafolio(pais, dolar_mep)

# Mostrar el DataFrame procesado
print(f"Portafolio procesado de {pais}:")
df_portafolio_procesado

## PARA VER TENENCIAS VALORIZADAS EN USD MEP Y GRAFICO DE TORTA

In [ ]:
#Divide el dataframe por tipo de activo, agrega la columna "Valorizado USD MEP" y si la moneda es "peso_Argentino
#divide la columna "Valorizado" por el valor del USD MEP. Toma por argumento el DataFrame con los activos y devuelve un diccionario 
#con DataFrames para cada tipo de activo
def procesar_activos_por_tipo(df_activos, dolar_mep):
    #Crear un diccionario para almacenar los DataFrames por tipo de activo
    activos_por_tipo = {}
    #Sumar los valores de valorización total
    total_valorizado_general = 0
    #Itero sobre cada tipo de activo único
    for tipo in df_activos["Tipo de Activo"].unique():
        #Filtro el DataFrame por tipo de activo
        df_tipo = df_activos[df_activos["Tipo de Activo"] == tipo].copy()
        #Agrego la columna "Valorizado USD MEP" y realizar la conversión si es necesario
        df_tipo["Valorizado USD MEP"] = df_tipo.apply(
            lambda row: row["Valorizado"] / dolar_mep if row["Moneda"] == "peso_Argentino" else row["Valorizado"], axis=1)
        #Agrego el DataFrame filtrado al diccionario
        activos_por_tipo[tipo] = df_tipo
        #Imprimir la sumatoria de la columna "Valorizado USD MEP" para cada tipo de activo
        total_valorizado = df_tipo["Valorizado USD MEP"].sum()
        print(f"Total valorizado {tipo} USD-MEP: {total_valorizado:.2f}")
        #Sumar al total general
        total_valorizado_general += total_valorizado
    # Imprimir la suma total de valorización
    print(f"Total valorizado general USD-MEP: {total_valorizado_general:.2f}")
    return activos_por_tipo

#Procesar los activos y obtener los DataFrames por tipo
activos_por_tipo = procesar_activos_por_tipo(df_activos, dolar_mep)

#Crear un grafico de torta. Argumento: diccionario con DataFrames
def graficar_torta_valorizado(activos_por_tipo):
    # Crear una lista con los totales valorizados para cada tipo de activo
    tipos = []
    totales_valorizados = []
    for tipo, df_tipo in activos_por_tipo.items():
        total_valorizado = df_tipo["Valorizado USD MEP"].sum()
        tipos.append(tipo)
        totales_valorizados.append(total_valorizado)
    #Crear el gráfico de torta
    plt.figure(figsize=(6, 6)) #Tamaño del grafico (600x600 por defecto)
    plt.pie(totales_valorizados, labels=tipos, autopct="%1.1f%%", startangle=90, colors=plt.cm.Paired.colors)
    plt.title("Distribución del Total Valorizado en USD MEP por Tipo de Activo", fontsize=14)
    plt.axis("equal")  #Igualar el aspecto para un círculo perfecto
    plt.show()

#Llamo la funcion para graficar
graficar_torta_valorizado(activos_por_tipo)

## DISTRIBUCION DE TENENCIAS POR SIMBOLO

In [ ]:
#Muuestra un grafico de barras con las valorizaciones en USD MEP de cada simbolo agrupado por tipo de activo
#Por argumento, diccionario con DataFrames de activos por tipo.
def graficar_barras_valorizacion(activos_por_tipo):
    plt.figure(figsize=(12, 8))
    #Recorrer cada tipo de activo en el diccionario
    for tipo, df_tipo in activos_por_tipo.items():
        # Obtener los símbolos y sus respectivas valorizaciones en USD MEP
        simbolos = df_tipo["Simbolo"].apply(lambda x: x.get("simbolo") if isinstance(x, dict) else x)
        valorizados_usd_mep = df_tipo["Valorizado USD MEP"]
        plt.bar(simbolos, valorizados_usd_mep, label=tipo, alpha=0.7)
    #Ajustar etiquetas y título
    plt.xlabel("Símbolo", fontsize=12)
    plt.ylabel("Valorizado USD MEP", fontsize=12)
    plt.title("Valorización en USD MEP", fontsize=14)
    plt.xticks(rotation=90)  # Rotar las etiquetas en el eje x para mejor visualización
    plt.legend(title="Tipo de Activo")
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()  # Ajustar el diseño para que no se corten las etiquetas
    plt.show()

# Llamar a la función para graficar las barras
graficar_barras_valorizacion(activos_por_tipo)

# BONUS: VER ACTIVOS SEPARADOS POR TIPO

In [ ]:
#Visualiza un DataFrame para uun tipo de activo en particular.
#Cambiar la variable "activo" de CEDEARS al activo deseado (CEDEARS, TitulosPublicos, ObligacionesNegociables, ACCIONES, 
#FondoComundeInversion, Letras)
activo = "CEDEARS"
activos_por_tipo.get(activo, None)